In [1]:
import sys; sys.path.insert(0,'..')
from geotaste import *
from pointpats import ripley
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist

In [2]:
df1 = get_mapdf(year=1920, event='borrow')
df2 = get_mapdf(year=1930, event='borrow')

len(df1), len(df2)

(5982, 12015)

In [3]:
df1

,member_id,book_id,event_type,start_date,end_date,member_uris,member_names,member_sort_names,subscription_price_paid,subscription_deposit,...,event_count,borrow_count,purchase_count,circulation_years,updated_book,start_year,start_dec,first_coordinates,lat,lon
event_id,,,,,,,,,,,,,,,,,,,,,
linossier-raymonde BORROWS norris-shanghaied ON 1920-12,linossier-raymonde,norris-shanghaied,Borrow,1920-12,,https://shakespeareandco.princeton.edu/members...,Raymonde Linossier,"Linossier, Raymonde",,,...,9,9,0,1920;1922;1923;1927;1941;1944;1947,2021-11-06T12:33:22.945119+00:00,1920,1920,"48.58826, 7.76566",48.58826,7.76566
wyck BORROWS norris-shanghaied FROM 1922-01-03 TO 1922-01-04,wyck,norris-shanghaied,Borrow,1922-01-03,1922-01-04,https://shakespeareandco.princeton.edu/members...,William van Wyck,"van Wyck, William",,,...,9,9,0,1920;1922;1923;1927;1941;1944;1947,2021-11-06T12:33:22.945119+00:00,1922,1920,"48.84835, 2.34269",48.84835,2.34269
milhaud BORROWS norris-shanghaied FROM 1922-07-04 TO 1922-10-20,milhaud,norris-shanghaied,Borrow,1922-07-04,1922-10-20,https://shakespeareandco.princeton.edu/members...,Madeleine Milhaud,"Milhaud, Madeleine",,,...,9,9,0,1920;1922;1923;1927;1941;1944;1947,2021-11-06T12:33:22.945119+00:00,1922,1920,"48.88237, 2.33871",48.88237,2.33871
wood-thelma BORROWS norris-shanghaied FROM 1923-09-17 TO 1923-09-26,wood-thelma,norris-shanghaied,Borrow,1923-09-17,1923-09-26,https://shakespeareandco.princeton.edu/members...,Thelma Wood,"Wood, Thelma",,,...,9,9,0,1920;1922;1923;1927;1941;1944;1947,2021-11-06T12:33:22.945119+00:00,1923,1920,"48.84192, 2.32767",48.84192,2.32767
joyce-lucia BORROWS norris-shanghaied FROM 1922-12-20 TO 1923-05-31,joyce-lucia,norris-shanghaied,Borrow,1922-12-20,1923-05-31,https://shakespeareandco.princeton.edu/members...,Lucia Joyce,"Joyce, Lucia",,,...,9,9,0,1920;1922;1923;1927;1941;1944;1947,2021-11-06T12:33:22.945119+00:00,1922,1920,"43.28911, 5.37434",43.28911,5.37434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
antoine-may BORROWS fletcher-double-chance FROM 1929-06-17 TO 1929-07-17,antoine-may,fletcher-double-chance,Borrow,1929-06-17,1929-07-17,https://shakespeareandco.princeton.edu/members...,Renée Antoine-May,"Antoine-May, Renée",,,...,2,2,0,1929;1932,2020-04-30T19:40:20+00:00,1929,1920,"48.84899, 2.33354",48.84899,2.33354
wigram BORROWS seiss-golden-altar-forms FROM 1929-05-13 TO 1929-06-07,wigram,seiss-golden-altar-forms,Borrow,1929-05-13,1929-06-07,https://shakespeareandco.princeton.edu/members...,"Ava (Bodley) Wigram, Viscountess Waverley / Mr...","Wigram, Ava",,,...,1,1,0,1929,2021-06-22T02:11:30+00:00,1929,1920,"48.86937, 2.31952",48.86937,2.31952
hobhouse BORROWS bercovici-story-gypsies ON 1929-07-17,hobhouse,bercovici-story-gypsies,Borrow,1929-07-17,,https://shakespeareandco.princeton.edu/members...,U. Hobhouse,"Hobhouse, U.",,,...,1,1,0,1929,2020-04-30T19:40:20+00:00,1929,1920,"48.84275, 2.33104",48.84275,2.33104


In [110]:
draw_heatmap(df1)

In [111]:
draw_heatmap(df2)

In [112]:
def get_points(df): 
    if type(df)==np.ndarray: return df
    return np.array([
        utm.from_latlon(lat,lon)[:2]
        for lat,lon in zip(df.lat, df.lon)
    ])
        

In [113]:
# get_points(df1.sample(n=100))

In [114]:
def measure_dist_points(df1, df2):
    from fastdist import fastdist    
    
    mindf = min([len(df1), len(df2)])
    
    df1s=df1.sample(n=mindf)
    df2s=df2.sample(n=mindf)
    points1=get_points(df1s)
    points2=get_points(df2s)
    dists=fastdist.matrix_to_matrix_distance(points1, points2, fastdist.euclidean, "euclidean")
    
    return pd.DataFrame(dists, columns=df2s.index, index=df1s.index)

In [115]:
dists=measure_dist_points(df1,df2)
dists.shape

(5982, 5982)

In [116]:
comparisons = [
    (dict(year=1920), dict(year=1930)),
    (dict(year=1930), dict(year=1940)),
    (dict(year=1940), dict(year=1950)),
    (dict(book='joyce-ulysses'), dict(book='woolf-mrs-dalloway')),
    (dict(book='joyce-portrait-artist-young'), dict(book='joyce-ulysses')),
    (dict(book='joyce-portrait-artist-young', year=1920), dict(book='joyce-portrait-artist-young', year=1930))
]

In [117]:
for d1,d2 in comparisons:
    df1=get_mapdf(**d1)
    df2=get_mapdf(**d2)
    dists=measure_dist_points(df1,df2)
    medians=(dists.median()/1000)
    n1=' '.join(str(x) for x in d1.values())
    n2=' '.join(str(x) for x in d2.values())
    med,std=medians.median(),medians.std()
    print(f'{med:.4}km ± {std:.4}km is the median distance between "{n1}" and "{n2}".')

1.995km ± 64.8km is the median distance between "1920" and "1930".
2.827km ± 51.05km is the median distance between "1930" and "1940".
2.617km ± 67.9km is the median distance between "1940" and "1950".
2.498km ± 2.855km is the median distance between "joyce-ulysses" and "woolf-mrs-dalloway".
3.264km ± 54.11km is the median distance between "joyce-portrait-artist-young" and "joyce-ulysses".
3.217km ± 180.6km is the median distance between "joyce-portrait-artist-young 1920" and "joyce-portrait-artist-young 1930".


In [102]:
# 1.953km ± 65.04km is the median distance between "1920" and "1930".
# 2.854km ± 51.04km is the median distance between "1930" and "1940".
# 2.439km ± 68.03km is the median distance between "1940" and "1950".
# 2.498km ± 2.77km is the median distance between "joyce-ulysses" and "woolf-mrs-dalloway".
# 2.901km ± 54.4km is the median distance between "joyce-portrait-artist-young" and "joyce-ulysses".

In [120]:
draw_heatmap(get_mapdf(book='joyce-portrait-artist-young', year=1920))

In [121]:
draw_heatmap(get_mapdf(book='joyce-portrait-artist-young', year=1930))

{'book': 'joyce-portrait-artist-young'}